<a href="https://colab.research.google.com/github/Lois-Wong/cs-course-recommender/blob/main/cs_course_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import logging
import requests
import re
import sys
from urllib import parse, request
from urllib.parse import urlparse
import heapq
import pandas as pd

### Public Access Computer Science Course Database


Run the code block below to view an expandable list of all courses in the database. Courses sources from the MIT OpenCourseware website and Coursera.

In [ ]:
! pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from sentence_transformers import SentenceTransformer, util


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
courses = pd.read_csv("https://raw.githubusercontent.com/Lois-Wong/cs-course-recommender/main/all_courses.csv")
courses = courses.drop(columns=courses.columns[:1])
for index, value in enumerate(courses["Description"]):
    # Example editing: appending " - edited" to each value
    courses.at[index, 'Description'] = value.replace("\n", "")


# Increment all row numbers by 1
courses.index += 1

import ipywidgets as widgets
from IPython.display import display, HTML

# Set the display options for pandas
pd.set_option('display.max_colwidth', None)  # Display full text in each cell

# Define initial display parameters
start_index = 10
rows_per_page = 10

# Define the function to display more rows
def show_more_rows(button):
    global start_index
    with output:
        end_index = start_index + rows_per_page
        if end_index >= len(courses):
            end_index = len(courses)
            button.disabled = True
        display(courses.iloc[start_index:end_index])
        start_index = end_index

# Display the second set of ten rows (rows 10-19)
display(courses.iloc[0:10])

# Create a button widget to show more rows
button = widgets.Button(description="Show More Rows")
button.on_click(show_more_rows)

# Create an output widget to display the DataFrame
output = widgets.Output(layout=widgets.Layout(width='100%', height='100%', overflow='scroll'))

# Display the button widget
display(output)
display(button)

,Title,Description,Link,Source
1,Mathematics for Computer Science,"This course covers elementary discrete mathematics for computer science and engineering. It emphasizes mathematical definitions and proofs as well as applicable methods. Topics include formal logic notation, proof methods; induction, well-ordering; sets, relations; elementary graph theory; integer congruences; asymptotic notation and growth of functions; permutations and combinations, counting principles; discrete probability. Further selected topics may also be covered, such as recursive definition and structural induction; state machines and invariants; recurrences; generating functions.",https://ocw.mit.edu/courses/6-042j-mathematics-for-computer-science-fall-2010/,MIT OpenCourseware
2,Mathematics for Computer Science,"This is an introductory course in Discrete Mathematics oriented toward Computer Science and Engineering. The course divides roughly into thirds:Fundamental Concepts of Mathematics: Definitions, Proofs, Sets, Functions, RelationsDiscrete Structures: Modular Arithmetic, Graphs, State Machines, CountingDiscrete Probability TheoryA version of this course from a previous term was also taught as part of the Singapore-MIT Alliance (SMA) programme as course number SMA 5512 (Mathematics for Computer Science).",https://ocw.mit.edu/courses/6-042j-mathematics-for-computer-science-fall-2005/,MIT OpenCourseware
3,Mathematics for Computer Science,"This subject offers an interactive introduction to discrete mathematics oriented toward computer science and engineering. The subject coverage divides roughly into thirds:Fundamental concepts of mathematics: Definitions, proofs, sets, functions, relations.Discrete structures: graphs, state machines, modular arithmetic, counting.Discrete probability theory.On completion of 6.042J, students will be able to explain and apply the basic methods of discrete (noncontinuous) mathematics in computer science. They will be able to use these methods in subsequent courses in the design and analysis of algorithms, computability theory, software engineering, and computer systems.This course is part of the Open Learning Library, which is free to use. You have the option to sign up and enroll in the course if you want to track your progress, or you can view and use all the materials without enrolling.",https://ocw.mit.edu/courses/6-042j-mathematics-for-computer-science-spring-2015/,MIT OpenCourseware
4,Computational Cognitive Science,"This course is an introduction to computational theories of human cognition. Drawing on formal models from classic and contemporary artificial intelligence, students will explore fundamental issues in human knowledge representation, inductive learning and reasoning. What are the forms that our knowledge of the world takes? What are the inductive principles that allow us to acquire new knowledge from the interaction of prior knowledge with observed data? What kinds of data must be available to human learners, and what kinds of innate knowledge (if any) must they have?",https://ocw.mit.edu/courses/9-66j-computational-cognitive-science-fall-2004/,MIT OpenCourseware
5,Great Ideas in Theoretical Computer Science,"This course provides a challenging introduction to some of the central ideas of theoretical computer science. It attempts to present a vision of “computer science beyond computers”: that is, CS as a set of mathematical tools for understanding complex systems such as universes and minds. Beginning in antiquity—with Euclid’s algorithm and other ancient examples of computational thinking—the course will progress rapidly through propositional logic, Turing machines and computability, finite automata, Gödel’s theorems, efficient algorithms and reducibility, NP-completeness, the P versus NP problem, decision trees and other concrete computational models, the power of randomness, cryptography and one-way functions, computational theories of learning, interactive proofs, and quantum computing and the phys

Output(layout=Layout(height='100%', overflow='scroll', width='100%'))

Button(description='Show More Rows', style=ButtonStyle())

### Search the database
Run the code block below and enter a search keyword/phrase token. The title and description of each course will be searched for the token you entered. All matches will be outputted as an expandable list. Run this block again to do another search.

In [ ]:
import pandas as pd


# Function to filter rows based on input token in title or description
def filter_rows(input_token):
    filtered_rows = pd.DataFrame(columns=courses.columns)
    for index, row in courses.iterrows():
        title = str(row['Title']).lower()
        description = str(row['Description']).lower()
        # Check if the input token is present in title or description
        if input_token.lower() in title or input_token.lower() in description:
            filtered_rows.loc[len(filtered_rows)] = row
    return filtered_rows


# User input for token
input_token = input("Enter a token: ")

# Filter rows based on input token
filtered_courses = filter_rows(input_token)

filtered_courses.index += 1

if len(filtered_courses) > 0:
    pd.set_option('display.max_colwidth', None)  # Display full text in each cell

    # Define initial display parameters
    start_index = 10
    rows_per_page = 10

    # Define the function to display more rows
    def show_more_rows(button):
        global start_index
        with output:
            end_index = start_index + rows_per_page
            if end_index >= len(filtered_courses):
                end_index = len(filtered_courses)
                button.disabled = True
            display(filtered_courses.iloc[start_index:end_index])
            start_index = end_index

# Display the second set of ten rows (rows 10-19)
    display(filtered_courses.iloc[0:10])

# Create a button widget to show more rows
    button = widgets.Button(description="Show More Rows")
    button.on_click(show_more_rows)

# Create an output widget to display the DataFrame
    output = widgets.Output(layout=widgets.Layout(width='100%', height='100%', overflow='scroll'))

# Display the button widget
    display(output)
    display(button)
else:
    print("No matches")

Enter a token: database


,Title,Description,Link,Source
1,Database Systems,"This course relies on primary readings from the database community to introduce graduate students to the foundations of database systems, focusing on basics such as the relational algebra and data model, schema normalization, query optimization, and transactions. It is designed for students who have taken 6.033 (or equivalent); no prior database experience is assumed, though students who have taken an undergraduate course in databases are encouraged to attend.",https://ocw.mit.edu/courses/6-830-database-systems-fall-2010/,MIT OpenCourseware
2,Software Studio,"This course on software engineering covers design and implementation of medium-scale software systems, using web applications as a platform. In the course, students learn the fundamentals of structuring a web application and writing modular code, with an emphasis on conceptual design to achieve clarity, simplicity, and modularity. Topics also include functional programming, relational databases, and security.",https://ocw.mit.edu/courses/6-170-software-studio-spring-2013/,MIT OpenCourseware
3,Information Technology (IT) Fundamentals for Everyone,"This course will guide you through fundamental concepts and skills to confidently navigate various aspects of IT and the digital world. You will be introduced to the vast field of IT, covering a wide range of topics to build your IT knowledge. You will delve into hardware, operating systems technology, software, programming, databases, networking, storage, cybersecurity, cloud computing, and support and operations. Designed to summarize the IT Fundamentals and Cloud specialization courses, this course is ideal for individuals seeking non-IT practitioner roles, such as project managers. It provides essential insights into the IT field, equipping you to collaborate effectively with IT teams and make informed decisions within your role. Each lesson contains materials such as hands-on labs and interactive activities to practice your newfound skills, glossaries to help you learn new vocabulary, as well as practice and graded quizzes to assess what you’ve learned.If you seek a non-technical role in the IT industry, this course provides you with a strong foundation so you can thrive in the ever-evolving IT landscape.",https://www.coursera.org/learn/information-technology-it-fundamentals-for-everyone,Coursera
4,"Introduction to Software, Programming, and Databases","Take the next step in your IT professional journey with this self-paced, beginner friendly course! This course covers the basics of software, cloud computing, web browsers, application development concepts, programming languages, and database fundamentals. You will first be introduced to computing platforms and software applications. You’ll learn about the software lifecycle, software compatibility across different platforms, and software versioning. You'll also learn about installing and managing web browsers, using extensions and plug-ins, and keeping them secure and updated. Next you will learn about basic programming concepts, coding logic, and the different programming languages like Python. You’ll move on to data and database fundamentals and learn how to use, interface with, access and backup databases. After completing this course, you'll have a better grasp of software processes and feel more confident about using and securing applications. From mobile phone operating systems to workplace database applications, this course covers a wide range of software. Discover the benefits of cloud-based technologies and see how using the cloud can help businesses create and deploy applications faster. To solidify your understanding and put your skills to the test, the course includes numerous hands-on labs. It ends with a final project where you’ll showcase your newfound expertise and demonstrate your understanding of software, programming, and databases. Enroll now to take the next step in developing expertise in IT Fundamental

Output(layout=Layout(height='100%', overflow='scroll', width='100%'))

Button(description='Show More Rows', style=ButtonStyle())

### Finding courses that suit your goals

In [ ]:

# Load a pre-trained sentence transformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Course descriptions
course_descriptions = [...]  # Your list of course descriptions

# Encode the course descriptions
course_embeddings = []
for entry in courses.itertuples():
    # Encode the course description
    course_description_embedding = model.encode(entry.Description, convert_to_tensor=True)
    course_embeddings.append(course_description_embedding)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#### Instructions

Run the code block below and enter a query that describes your goals for your CS education. After you enter your query, a sorted list of the courses that might best suit your goals will appear. Run the code block again to enter a different query.

In [ ]:
# User input
user_goals = input("What are your goals for an online CS course? ")
print("Loading...")

# Encode the user's goals
user_goals_embedding = model.encode(user_goals, convert_to_tensor=True)

# Calculate similarity scores for each course description
similarity_scores = []
for i, entry in enumerate(courses.itertuples()):
    # Use the precomputed course description embedding
    course_description_embedding = course_embeddings[i]

    # Calculate the similarity between the embeddings
    cosine_score = util.pytorch_cos_sim(user_goals_embedding, course_description_embedding).item()

    # Store the similarity score along with the corresponding course description
    similarity_scores.append((cosine_score, entry.Title, entry.Description, entry.Link, entry.Source))

# Sort the courses based on similarity scores in descending order
similarity_scores.sort(key=lambda x: x[0], reverse=True)

# Output the top ten most similar courses in a dataframe
df = pd.DataFrame(similarity_scores, columns=['Similarity Score', 'Title', 'Description', 'Link', 'Source'])

import ipywidgets as widgets
from IPython.display import display, HTML

# Set the display options for pandas
pd.set_option('display.max_colwidth', None)  # Display full text in each cell

# Define initial display parameters
start_index = 10
rows_per_page = 10

# Define the function to display more rows
def show_more_rows(button):
    global start_index
    with output:
        end_index = start_index + rows_per_page
        if end_index >= len(df):
            end_index = len(df)
            button.disabled = True
        display(df.iloc[start_index:end_index])
        start_index = end_index

# Display the second set of ten rows (rows 10-19)
display(df.iloc[0:10])

# Create a button widget to show more rows
button = widgets.Button(description="Show More Rows")
button.on_click(show_more_rows)

# Create an output widget to display the DataFrame
output = widgets.Output(layout=widgets.Layout(width='100%', height='100%', overflow='scroll'))

# Display the button widget
display(output)
display(button)

What are your goals for an online CS course? I want to learn how to build a chatbot
Loading...


,Similarity Score,Title,Description,Link,Source
0,0.699993,Building AI Powered Chatbots Without Programming,"This course will teach you how to create useful chatbots without the need to write any code. Leveraging IBM Watson's Natural Language Processing capabilities, you'll learn how to plan, implement, test, and deploy chatbots that delight your users, rather than frustrate them.True to our promise of not requiring any code, you'll learn how to visually create chatbots with Watson Assistant (formerly Watson Conversation) and how to deploy them on your own website through a handy WordPress plugin. Don't have a website? No worries, one will be provided to you.Chatbots are a hot topic in our industry and are about to go big. New jobs requiring this specific skill are being added every day, consultants demand premium rates, and the interest in chatbots is quickly exploding.Gartner predicts that by 2020, 85% of customer interactions with the enterprise will be through automated means (that's chatbots and related technologies).Here is your chance to learn this highly in demand set of skills with a gentle introduction to the topic that leaves no stone unturned.",https://www.coursera.org/learn/building-ai-powered-chatbots,Coursera
1,0.554569,Learn to code with AI,"Imagine waking up tomorrow as a web developer. What would you want to build? With AI tools like ChatGPT, you're already a developer, regardless of your experience, if you know how to work with them.So in this course, you'll build functional, interactive front-end projects while learning how to write effective prompts and debug and refine your code with the help of AI.No coding experience needed! We'll focus on helping you prototype and build projects with AI's assistance, turning you from a non-coder into a capable problem solver.By the end of this course, you'll have a collection of mini-projects, newly acquired skills, and a solid foundation to keep building with AI.You'll work on various projects using HTML, CSS, and JavaScript. Let's do this!",https://www.coursera.org/learn/learn-to-code-with-ai,Coursera
2,0.523536,Learn HTML and CSS,"This course is perfect if you want to learn how to code websites from scratch. You will learn the basics of HTML and CSS through interactive challenges. Along the way, you will build multiple projects, like a Google.com clone, a Space Exploration page, and a fun birthday site filled with GIFs. While most of the challenges are done in Scrimba's interactive editor, you will also learn how to use professional tools like VS Code for code editing, GitHub for hosting your code, and Netlify for deploying your projects to the world wide web.",https://www.coursera.org/learn/learn-html-and-css,Coursera
3,0.512024,API Development on Google Cloud's Apigee API Platform,"In this course, you learn how to create APIs that utilize multiple services and how you can use custom code on Apigee. You will also learn about fault handling, and how to share logic between proxies. You learn about traffic management and caching. You also create a developer portal, and publish your API to the portal. You learn about logging and analytics, as well as CI/CD and the different deployment models supported by Apigee. Through a combination of lectures, hands-on labs, and supplemental materials, you will learn how to design, build, secure, deploy, and manage API solutions using Google Cloud's Apigee API Platform.This is the third and final course of the Developing APIs with Google Cloud's Apigee API Platform course series.",https://www.coursera.org/learn/api-development-apigee-gcp,Coursera
4,0.488195,Programming with Scratch,"Learning coding is not only about understanding the programming language being used, but also developing important computational thinking skills, which are useful for problem solving across many disciplinary areas. In this course, students will learn basic programming skills by creating interactive storybooks, animations, and games with Scratch, which is

Output(layout=Layout(height='100%', overflow='scroll', width='100%'))

Button(description='Show More Rows', style=ButtonStyle())